https://github.com/fastai/courses/blob/master/deeplearning1/nbs/char-rnn.ipynb

In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
%matplotlib inline
import importlib
import matplotlib.pyplot as plt
import matplotlib.ticker
import numpy as np
import random
import time
import os
import pandas as pd
import csv
import math
import bcolz
import pickle
import re
import pathlib
import datetime

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable

SOS_TOKEN = '<SOS>' # Start Of Word

# Data setup

In [3]:
# importing keras might cause problems with cudann version etc
# import keras # some good utils in here
# path = keras.utils.data_utils.get_file('nietzsche.txt', origin="https://s3.amazonaws.com/text-datasets/nietzsche.txt")

In [4]:
path = '/home/ohu/.keras/datasets/nietzsche.txt'
text = open(path).read().lower()
path
print('corpus length:', len(text))

'/home/ohu/.keras/datasets/nietzsche.txt'

corpus length: 600893


In [5]:
!tail {path} -n5

not to be regarded as a pure type. Knowledge and science--as far as they
existed--and superiority to the rest of mankind by logical discipline
and training of the intellectual powers were insisted upon by the
Buddhists as essential to sanctity, just as they were denounced by the
christian world as the indications of sinfulness.

In [6]:
chars = sorted(list(set(text)))
print('total chars:', len(chars))
''.join(chars)

total chars: 57


'\n !"\'(),-.0123456789:;=?[]_abcdefghijklmnopqrstuvwxyzäæéë'

In [7]:
char_indices = dict((c, i) for i, c in enumerate(chars))
char_indices['\n']

0

# Utils

In [8]:
def string_to_tensor(in_str, chars_index=char_indices, as_variable=True):
    """Onehot encoded tensor of string"""
    tensor_length = len(in_str)
    tensor = torch.zeros(1, tensor_length, len(chars_index))
    for li, letter in enumerate(in_str):
        tensor[0, li, chars_index[letter]] = 1
    if as_variable:
        tensor = Variable(tensor).cuda()
    return tensor

string_to_tensor('hello', as_variable=False).size()

torch.Size([1, 5, 57])

In [9]:
def time_since(since):
    now = time.time()
    s = now - since
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

# Char Rnn

In [10]:
class CharRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(CharRNN, self).__init__()
        
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.layers = 1
        
        self.rnn = nn.GRU(input_size, hidden_size, batch_first=True)
        self.lin_output = nn.Linear(hidden_size, input_size)
        
        # self.log_softmax = nn.LogSoftmax(dim=1) # current releas doesn't yet support dimensions
        
    def forward(self, input_char_vs, hidden = None):
        batch_size = input_char_vs.size()[0]
        if hidden is None:
            hidden = self.init_hidden(batch_size)
        
        rnn_outputs, hidden = self.rnn(input_char_vs, hidden)
        
        outputs = self.lin_output(rnn_outputs)
        #outputs = self.log_softmax(outputs)
        
        #outputs = F.log_softmax(outputs[0])

        return outputs, hidden

    def init_hidden(self, batch_size=1):
        return Variable(torch.zeros(self.layers, batch_size, self.hidden_size)).cuda()
    
model = CharRNN(len(chars), 512).cuda()
model

CharRNN (
  (rnn): GRU(57, 512, batch_first=True)
  (lin_output): Linear (512 -> 57)
)

In [11]:
#tmp = Variable(torch.FloatTensor([2]).view(1,1,-1)).cuda()
#tmp = Variable(string_to_tensor('hello')).cuda()
tmp = model(string_to_tensor('hello'))
len(tmp[0][0])
tmp[0].size()
tmp[1].size()
#chars[tmp[0].topk(1)[1].data[0][0]]

5

torch.Size([1, 5, 57])

torch.Size([1, 1, 512])

In [12]:
tmp = torch.cat((string_to_tensor('hello'), string_to_tensor('hello')))
#tmp.size()

tmp = model(tmp)
tmp[0].size()
tmp[1].size()

torch.Size([2, 5, 57])

torch.Size([1, 2, 512])

# Eval

In [34]:
# This is faster that uses the existing state untill the end
def print_example(iters=320, choice=True):
    seed_string="ethics is a basic foundation of all that"
    result_string = ''
    hidden = model.init_hidden()
    output, hidden = model(string_to_tensor(seed_string), hidden)
    
    for i in range(iters):
        output = output[0, -1]
        if choice:
            next_char = np.random.choice(chars, p=F.softmax(output).data.cpu().numpy())
        else:
            next_char_idx = output.topk(1)[1].data[0] 
            next_char = chars[next_char_idx]
        
        result_string += next_char
        output, hidden = model(string_to_tensor(next_char), hidden)
    print(result_string)
tmp = 200
print_example(iters=tmp, choice=True)
print('------------------')
print_example(iters=tmp, choice=False)

 would in
onde that? must_!"
118. he wo
long with whatever plytol some has intruestion, religion of molation in which they arrein musting ones, erammatic mexty dosing them. buture altechated, will by 
------------------
 is the strength of the strength of the strength of the strength of the strength of the strength of the strength of the strength of the strength of the strength of the strength of the strength of the 


# Samples

In [16]:
def get_random_sample_string(length=41):
    sample_place = random.randint(0, len(text)-length-1)
    sample = text[sample_place:sample_place+length]
    return sample
get_random_sample_string()

'essing the ground, are the occasion of ot'

In [17]:
sample_size = 50000

sample_sentence_size = 40
sample_data = torch.zeros(sample_size, sample_sentence_size, len(chars)) # .cuda()
#sample_target = torch.zeros((sample_size, sample_sentence_size, 1), torch.LongTensor)
sample_target = torch.LongTensor(sample_size, sample_sentence_size).zero_() # .cuda()
for i in range(sample_size):
    sample = get_random_sample_string(sample_sentence_size+1)
    sample = [char_indices[c] for c in sample]
    for j in range(sample_sentence_size):
        sample_data[i][j][sample[j]] = 1
        sample_target[i][j] = sample[j+1]

In [18]:
i = 5
j = 10
sample_data[i].topk(1)[1][j][0] == sample_target[i][j-1]
''.join([chars[c[0]] for c in sample_data[i].topk(1)[1]])

True

' "how are synthetic judgments a priori\np'

In [47]:
clip_parameters_value = 0.25

def train_single(optimizer, loss_function, batch_size=64):
    model.zero_grad()
    loss = 0
    output_lines = []
    
    sample_i = random.randint(0, len(sample_data)-1-batch_size)
    #x = sample_data[sample_i].view(1,40,len(chars))
    #y = sample_target[sample_i]
    
    x = sample_data[sample_i:sample_i+batch_size].view(batch_size,40,len(chars))
    y = sample_target[sample_i:sample_i+batch_size]
    
    target = Variable(y).cuda()

    hidden = model.init_hidden(batch_size)
    outputs, hidden = model(Variable(x).cuda(), hidden)

    top_chars = outputs.topk(1)[1].data
    for i in range(len(outputs)):
        loss += loss_function(outputs[i], target[i])       
        output_lines.append(''.join([chars[c[0]] for c in top_chars[i]]))
    
    if clip_parameters_value:
        torch.nn.utils.clip_grad_norm(model.parameters(), clip_parameters_value)
        
    loss.backward()
    optimizer.step()

    return output_lines, loss.data[0]/batch_size

tmp = train_single(torch.optim.Adam(model.parameters(), lr=0.0001), torch.nn.CrossEntropyLoss())
tmp[0][0:2]
tmp[1]

['ntrwer  ahich inlyged toe  th bavd ae or',
 'nr of the suestion  thes is aoat iuite i']

1.4231221675872803

In [48]:
%%timeit
train_single(torch.optim.Adam(model.parameters(), lr=0.001), torch.nn.CrossEntropyLoss(), batch_size=64)

44.6 ms ± 2.67 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [21]:
def train_iterations(n_iters=100000, lr=0.001, batch_size=64,
                     print_every=10000, plot_every=1000):
    
    def print_infos():
        print("{:>6d} {:>4.0%} ({:>8}) {:>7.3f}   | {:>6.2f}: {}".format(
          model_training.iterations, iteration/n_iters, time_since(start),
          current_loss/current_loss_iter, loss, result))
    
    
    global optimizer
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    
    start = time.time()
    model.train()

    current_loss = 0
    current_loss_iter = 1
    
    for iteration in range(1, n_iters + 1):
        model_training.iterations += 1
        
        #use_teacher_forcing = random.random() < teacher_forcing_ratio
        use_teacher_forcing = False
        
        #loss_function=nn.NLLLoss()
        loss_function = torch.nn.CrossEntropyLoss()
        result, loss = train_single(optimizer=optimizer, loss_function=loss_function, batch_size=batch_size)
        result = result[0]
        
        current_loss += loss
        current_loss_iter += 1

        # Print iter number, loss, name and guess
        if iteration % print_every == 0:
            print_infos()

        # Add current loss avg to list of losses
        if iteration % plot_every == 0:
            model_training.losses.append(current_loss / plot_every)
            model_training.learning_rates.append(lr)
            current_loss = 0
            current_loss_iter = 1

    print_infos()


In [22]:
from pytorch_utils_oh_3 import ModelTraining
MODEL_SAVE_PATH = 'char_rnn_fast_ai_testing_2'

Pytorch utils oh: pytorch_utils_oh_3.py
Pytorch: 0.2.0_4


In [23]:
model = CharRNN(len(chars), 512).cuda()
model
model_training = ModelTraining(MODEL_SAVE_PATH, [model])

CharRNN (
  (rnn): GRU(57, 512, batch_first=True)
  (lin_output): Linear (512 -> 57)
)

Save path: data/models/char_rnn_fast_ai_testing_2


In [24]:
train_iterations(n_iters=10, print_every=5)

     5  50% (   0m 0s)   3.297   |   3.82: s                                       
    10 100% (   0m 0s)   3.330   |   3.24: ss                                      
    10 100% (   0m 0s)   3.330   |   3.24: ss                                      


In [25]:
train_iterations(n_iters=10000//64, print_every=1000//64)

    25  10% (   0m 0s)   2.902   |   3.09: r t                                     
    40  19% (   0m 1s)   2.950   |   2.96: s                                       
    55  29% (   0m 2s)   2.940   |   2.84: rt  e                ee    e            
    70  38% (   0m 2s)   2.903   |   2.74:   tn tn   tn tn     t     en  the  tn   
    85  48% (   0m 3s)   2.856   |   2.60: nee an tn  aote e e thnheth e  ehahen  t
   100  58% (   0m 4s)   2.809   |   2.70: the then    tn tn an  tn then  an  ens e
   115  67% (   0m 4s)   2.769   |   2.51: eneetn sn   ao e toen aen ah te tor e n 
   130  77% (   0m 5s)   2.731   |   2.52: theneetf tn an the tf  nes ae e  aheneet
   145  87% (   0m 6s)   2.697   |   2.41: dtnd ao arn e  ne er erehneslsh and aor 
   160  96% (   0m 6s)   2.668   |   2.38:  toir  ahe toetl   ee af the t etl  an t
   166 100% (   0m 7s)   2.656   |   2.33: eteeng    tooen e   aorhetn an ter eete 


In [26]:
print_example()

ethics is a basic foundation of all that ara onsree co
ve bel themo on alvemeceseprithaciuly what
ust"-: ncend ise neis uhe af of enaverd thingbest ane,, hes derktead
oupsuwe worhentelssutisery me u4 sof egarly of hte ofcis the se fowbinlle the yton itsees
phe inest af of iod nle co
my cnofpinnt ir, of toall, to the iict helist ald ghedm cotns af onber and d


In [27]:
train_iterations(n_iters=10000//64, print_every=1000//64)

   181  10% (   0m 0s)   2.232   |   2.37: r rlt rt reinl   af trd re-ers  --n     
   196  19% (   0m 1s)   2.279   |   2.29: toeuethet  hree af th  e rion tn on an t
   211  29% (   0m 1s)   2.283   |   2.34:  ahe trenltn h e-n ert of terentorehnesl
   226  38% (   0m 2s)   2.283   |   2.28: n rlorer tor tett en  and ahe senhse nht
   241  48% (   0m 3s)   2.275   |   2.21: e th27  wfee t nn  tner nn  tum ns iott 
   256  58% (   0m 3s)   2.269   |   2.29: hhet tetitn  ysh  thenh thtd  ah tnleest
   271  67% (   0m 4s)   2.263   |   2.21:  tnd ae eng ah tof  tn eif  s  and   af 
   286  77% (   0m 5s)   2.254   |   2.18: hon tndorn  thith tnt ean andtn  nitn ly
   301  87% (   0m 5s)   2.246   |   2.14:   ahrl  aoet sn y aoeueth l   and r ahe 
   316  96% (   0m 6s)   2.236   |   2.19: e aotiiote  ahet totiielf antirn aomreea
   322 100% (   0m 6s)   2.232   |   2.16: nl   oh the poethe oritane  e netf txter


In [28]:
print_example()

ethics is a basic foundation of all that thes, if in aver--in s oxpriniteconio cajurel--aul ous entering the bote, oh ied's, andeonn" inperyule,
anctifer and andirnow, the tanted
-os thede thes mpals wiom vande" an iry maee lfacs of
ascatal of mastopi--) hatants of
aleverorf
thinasely breacs, andelo to hith gurtines mann prenturasien trat emingod souts.--nat


In [29]:
train_iterations(n_iters=100000//64, print_every=10000//64)

   478  10% (   0m 6s)   2.046   |   1.95: h heige  ond tn tienant   tn  aorderent 
   634  20% (  0m 14s)   1.976   |   1.83:      and tcemher tan  aor r an o tne cer
   790  30% (  0m 21s)   1.910   |   1.68: e ahe seing r ng trichologi af tormer th
   946  40% (  0m 28s)   1.852   |   1.63: the sxpentalltirld an aot the sarlnof tu
  1102  50% (  0m 35s)   1.803   |   1.63:   iateof trotefide tiich ts irsst oor e 
  1258  60% (  0m 41s)   1.761   |   1.53:  ul  aot inh tand ng toe ponrtge us aecn
  1414  70% (  0m 48s)   1.476   |   1.49: he snter tirld if tvpmted  avotionsl  ar
  1570  80% (  0m 55s)   1.468   |   1.52:  tonteryof tue s oorndairtues, aonrsges 
  1726  90% (   1m 3s)   1.454   |   1.40: e igions cn erpretation of tvpstence  ah
  1882 100% (  1m 10s)   1.440   |   1.37: hahet thes ionsint ttould anlays secannt
  1884 100% (  1m 10s)   1.440   |   1.39:  -w

9


the e are aon aho cre andgeudab


In [35]:
print_example()

 of i mainy in otser for hilplo's bory-nobadow
and decest-and pawn
highlound, and a former to fash it not said bo
known the troupter which rules--whether is how relesile!"_ pirit. every decedi-not permitting of mancessis and days can, these acts of insomethe on ics always which, kndwequanily lendity, these ramissions a


In [36]:
print_example()

 there is not foofryther propenty of man naster,"--our "froensmen har nite their precisely
accordingly hervelict and themselves: in everything of froend forthine; a non--he sedued,
shouls, but to is: the simely saym to believes of defined itself is whom--on omy pride of curto, when he censur histruas, paptristic to kno


In [37]:
print_example()

 subject authortaint knows, how cutialization, geniual myself out, fow r.-indeed,' hrances--all lised up consideraty?; ow "ciscivaling ouch utwarant
give what is gond also, or erroceatirnt: of a docaling spones and pirif antirmated to polithes of vilwer in reado "more
expeniences! the most free-simenicated with an whic


In [49]:
train_iterations(n_iters=100000//64, print_every=10000//64)

  2040  10% (   0m 7s)   1.358   |   1.33: tn  cwject ohe sndluetion of trin apon t
  2196  20% (  0m 13s)   1.351   |   1.34:   avough to tonrterence on ias tot tohop
  2352  30% (  0m 20s)   1.339   |   1.33:   testoose  -wn is anher ise thth the sa
  2508  40% (  0m 27s)   1.330   |   1.31:  r ioul  tn t robpen rsond aespoveredsth
  2664  50% (  0m 33s)   1.322   |   1.31: h rsf ttamtumes   ttmean ts tomserni tor
  2820  60% (  0m 40s)   1.314   |   1.30:  oometsastions of the mes   the moneroyw
  2976  70% (  0m 47s)   1.223   |   1.34: ertlere -onmery ppcuctive cntosphere -af
  3132  80% (  0m 54s)   1.231   |   1.10:  tn theycuestion wherher tty hing ounaph
  3288  90% (   1m 0s)   1.228   |   1.18:   ahe sxpeession  ot iente ff toral   in
  3444 100% (   1m 7s)   1.220   |   1.17:  orint  

44. ts at the fernlariporsa en
  3446 100% (   1m 7s)   1.220   |   1.15: the suestion  thes is titt buite isnored


In [50]:
print_example()

 is the abxistomed. who would still themselves from for those other mavadity.--only command nor before look readiness, granted of opon themselves that you
make its placting and
most frandate from them from himsple makes he
preserve to progous, and what science, in former naturally
by means on thiumsuld
so is selocker. 


In [51]:
print_example()

 which has just faith-in himself,
even under people must, the very older him.--therefore utulates a fundamental contemplation of the fivile, which scholars in this world will be ever been churd, and he happens to their done hit loves remains.  have been amenot have always, out loother hand misunderstandings, in order t


In [52]:
train_iterations(n_iters=100000//64, print_every=10000//64)

  3602  10% (   0m 6s)   1.183   |   1.08:  th these onteosers, theugh they way ben
  3758  20% (  0m 13s)   1.180   |   1.17: d rd  th e all tf them aesessantenmn toi
  3914  30% (  0m 20s)   1.174   |   1.20: -oass fofe asdexeryreasi sosgrast on eea
  4070  40% (  0m 27s)   1.166   |   1.10:   ry,ond tustice as iery dreat  ttpeliev
  4226  50% (  0m 33s)   1.157   |   1.13: e e  thit ws to say,tt.e dpirituoast be 
  4382  60% (  0m 40s)   1.152   |   1.05:  ind ohich iannure ohersorlhyof thesgs a
  4538  70% (  0m 47s)   1.094   |   1.08: tesrh as if the sosll  wt ething tnconmu
  4694  80% (  0m 53s)   1.091   |   1.10: n tesenn e if aissthet tu the sasis of w
  4850  90% (   1m 0s)   1.086   |   1.06: e  nt  ae wast tpder tond thet the seeat
  5006 100% (   1m 7s)   1.081   |   1.10: enturtes dn urpire- 
1ut toch aeluacs ty
  5008 100% (   1m 8s)   1.081   |   1.07: etes bone tld dn even lomeng oore and mo


In [53]:
print_example()

 serkembnes--and "christianity as attrmatual fact, the will to all philosophy and
shampless history is politin spaces at least is unavoud fortule;
from the scientific man!--is, for the fablicates of undellabing conditioned
with our feasen not necessary to dange further, thereflee, sesief rurely sould, as he chuscless c


In [54]:
print_example()

 attempt hours, of
the finest euterness the whole styeen, to become justice.=--philology
surplus of culture: for
their body, and men of
the flaurerde, moleress, and conceingumping to every own, what we clearly encounal-yough for wimn--understanding here, the party, to some plabour;--ad
slaid), and below the property, b


In [55]:
train_iterations(n_iters=100000//64, print_every=10000//64)

  5164  10% (   0m 6s)   1.042   |   1.08:  etr hiwever, ts toch mo  saow and seaep
  5320  20% (  0m 13s)   1.041   |   0.99: n tieorous growth --it iesects the cherr
  5476  30% (  0m 20s)   1.035   |   1.06: hich te no   the melsation of the eontit
  5632  40% (  0m 27s)   1.030   |   1.09: thit iasherto hpders thambrs  eet weot t
  5788  50% (  0m 33s)   1.026   |   1.00: n ng sis owservation fpon mxpeptional mh
  5944  60% (  0m 40s)   1.020   |   1.01: thet ian is lience  maniind hoes ne isfe
  6100  70% (  0m 47s)   0.974   |   0.98: n ihesks hemain ior the mreat  the mcsss
  6256  80% (  0m 54s)   0.978   |   1.08: e arseirnning and dotho  oxd, lake ise h
  6412  90% (   1m 0s)   0.975   |   1.01: h e alpewessity of tvfect  an t dard, te
  6568 100% (   1m 8s)   0.971   |   1.04: th toristianity an wly cirme for whth st
  6570 100% (   1m 8s)   0.971   |   0.91: trrn andact oay nasse another? as far as


In [56]:
print_example()

 is stormed, must be weazers in order that the "hravenenal spirits!

292. in havigg, ages, many-dishapsedein.


169

=the there of phusoness, such benof concersming, suptrmert hum unted things, that the contempt fremshlandered or somebody.

98. whoever has
dohe than only when it is the belief to them comes our prive at


In [57]:
print_example()

 words in misunderstlondection, we call
"higher, dreaming; it wad the religious men--orator, bod,
in the
former and action,
bying that go whot
pleasure
of thin
constrenges:
the word itself having at least, but syspech to the time of humanity (the real upon what is

uncertain, the begail and of view megraties at the box


In [58]:
train_iterations(n_iters=100000//64, print_every=10000//64)

  6726  10% (   0m 6s)   0.957   |   1.02:   toamsros could ell tb them fonesto us 
  6882  20% (  0m 13s)   0.952   |   0.91: iohe aore ooarply at that ps con in him,
  7038  30% (  0m 19s)   0.945   |   0.90: e ess,an tttersion, nor anything it once
  7194  40% (  0m 26s)   0.941   |   0.93:   ah ssl thet eevuces, all res  wonstrai
  7350  50% (  0m 33s)   0.938   |   0.99: e  tomes  ahe stul oad arown aeact  ahe 
  7506  60% (  0m 39s)   0.933   |   0.91:  aot bnow  and ehat ihe focous oetient a
  7662  70% (  0m 46s)   0.888   |   0.93: tive ro eelieve in tvernal life an drder
  7818  80% (  0m 52s)   0.904   |   0.90: heue mhoas which hequirescorturyes of mo
  7974  90% (  0m 59s)   0.900   |   0.85: iohivmyny  even similarity of view oint 
  8130 100% (   1m 6s)   0.899   |   0.97:  aast kortainly teal with an
ere to the 
  8132 100% (   1m 6s)   0.899   |   0.79: thrlsan,ike liborious ess  to whoch te h


In [59]:
print_example()

 goethe's "good conscience the trouble show such indiscourable and the spirit would
allured as naired," which is no
longer godo, dirocons itself is a religious aften, decey, such as survays happens narrew and does not
among at present danger, that "chilors sin even without her delusion great.=--the dogm so! be ornaboe 


In [60]:
print_example()

, of everything consciousness of has homines, frate. and by the world--and revengefulness" injury instruction, who are in the habit of patagnoos. there are hindred by ustlanders not believe that has been
bystally, to
zouth not to speak of
sknieitome"......

264. it may be incomprehensible one
sense on the day after to 


In [63]:
train_iterations(n_iters=100000//64, print_every=25000//64)

  8522  25% (  0m 16s)   0.873   |   0.77: tns sthe dard   tas hitherto bouumphed  
  8912  50% (  0m 33s)   0.870   |   0.69: h bhe pew pailosophers hho are atpearing
  9302  75% (  0m 50s)   0.844   |   0.91: ell   ahe sesiruction-of the cndividuals
  9692 100% (   1m 7s)   0.844   |   0.93:  e ning theyouelities of mank


42

=cla
  9694 100% (   1m 7s)   0.844   |   0.82: tii fxrher exgenders on froduces--buth w


In [64]:
print_example(iters=2000)

 immediate druak and lice
small, on thi

bit little a origin of as a matter of
fecrives incertional sufferings, when he is desired light and noneral..
perhaps the rightly
andisint, our seeknts of life, is hasteristic original state obetien, aristo thickly "given" "one may be a serean before the molier, all put itself looked upon as affownthe higher nature, impulse, conquest
of character," as their truth? an everything of the exemutical influence within them as indamed although the feelings to whom
relistening, from his wake him, as improst the things that exist, reverge them to liee and connected itself is a symptoms of another, and
task of its plyasity and of treather
want however, uncontious, and no matter of fact, the necessary consequences of enormously, and that has
not only about one ruler, the
designation and sense of truths ase
"inaxplicable that createvel, over-knowledge of
the community or a cestath to)
corpsivaling itility are declined and
display as the rest of
a majority a

In [65]:
print_example(iters=2000)


the old familiar
darid to a new
medicinating in garm and contains a pleasure is descener
crrations around not without degenerated hellening an antidote to laits
understood" in himself the control of a
man who certainly, no longer race. they are no moath out
mask-oneself, nature been anywhere, or conceal himself or a certain virtus does see should
come to rogative, and all philosophical successful experience to its
uncreasor and his belief in
woman"--supposing now the uspessimist spirituality and doctrine
of under painfus, imphended bud
becomes for this is a sort of contume form--conser"ning it rejosced morality stroke things remais superficial, neither stades. this assumptions puriod spitst german savisfue, more
dolities. among one--love hor
to have
long agryemetabieace--and why? in rave--the simple time to-day.

164. only interlocutors,
in shout has pethants, however, are nove the rise
tomenhe and lack of casef it.--"equality and dominated
him as the self absolubes doftheter be quati